In [1]:
import os
import numpy as np
import SimpleITK as sitk
from sitk2vtk import *
from utils import *

In [2]:
feature_angle = 120
pass_band = 0.001

material_set = {"steel": 1, "porosities": 2}

In [3]:
fname = os.path.join(base_dir, "segmentation", "Segment-980.mha")
segmented_volume_sitk = sitk.ReadImage(fname)

input_size = segmented_volume_sitk.GetSize()
input_spacing = segmented_volume_sitk.GetSpacing()
print("Image size:", input_size[0], input_size[1], input_size[2])
print("Image spacing:", input_spacing[0], input_spacing[1], input_spacing[2])

Image size: 1008 1008 980
Image spacing: 0.009996474264487798 0.009996474264487798 0.009996474264487798


In [4]:
# Pad the image
pad_filter = sitk.ConstantPadImageFilter()
pad_filter.SetConstant (0)
pad_filter.SetPadLowerBound ([1, 1, 1])
pad_filter.SetPadUpperBound ([1, 1, 1])
padded_image = pad_filter.Execute(segmented_volume_sitk)               

In [5]:
mesh_dir = os.path.join(base_dir, "meshes")
if not os.path.exists(mesh_dir):
    os.mkdir(mesh_dir)

In [6]:
for label in material_set:
    
    print("Process", label)
    
    threshold = material_set[label]

    # Threshold the phantom
    binary_image = (padded_image == threshold)

    # Smooth the binary segmentation
    print("\tStart smoothing")
    smoothed_binary_image = sitk.AntiAliasBinary(binary_image)
    del binary_image
    print("\tSmoothing done")
        
    # Create a VTK image
    vtkimg = sitk2vtk(smoothed_binary_image, centre=True)
    del smoothed_binary_image
    
    print("\tStart meshing")
    vtk_mesh, meshing_filter = extractSurface(vtkimg, 0)    
    mesh_fname = os.path.join(mesh_dir, label + ".stl")
    writeSTL(vtk_mesh, mesh_fname)
    del vtkimg
    print("\tMeshing done")
    
    print("\tSmooth meshing")
    vtk_smoothed_mesh = smoothSurface(meshing_filter, feature_angle=feature_angle, pass_band=pass_band)
    del vtk_mesh
    print("\tSmoothing done")

    mesh_fname = os.path.join(mesh_dir, label + "-smoothed.stl")
    writeSTL(vtk_smoothed_mesh, mesh_fname)
    del vtk_smoothed_mesh

Process steel
	Start smoothing
	Smoothing done
	Start meshing
	Meshing done
	Smooth meshing
	Smoothing done
Process porosities
	Start smoothing
	Smoothing done
	Start meshing
	Meshing done
	Smooth meshing
	Smoothing done


In [7]:
print("DONE")

DONE
